# Functional Programming for Data Analysis

### Jim Pivarski

Third notebook: Scala

Scala is a functional programming language, like Haskell but not as strict, and it runs on the Java Virtual Machine (JVM).

This last point makes it harder to integrate into physics applications, but easier to integrate into business analytics, since most computing infrastructure in industry is based on Java, rather than C++.

Scala is also Spark's native tongue: Spark was written in Scala and provides Java, Python, and R interfaces as a convenience.

<img src="https://spark.apache.org/images/spark-logo-trademark.png" style="width: 400px; margin-left: auto; margin-right: auto;">

Python programming in Spark is not efficient. If you're going to be doing any architectural work in Spark, you should use Scala.

Scala also provides an example of type-safe functional programming, which has better error messages and safety when building large applications.

It also has pattern-matching, which my "functional playground" in Python lacks.

In [48]:
// Crash course in Scala syntax

val xs = List(1, 2, 3, 4, 5)    // statically typed, but inferred (like C++'s "auto")

val ys = 1 :: 2 :: 3 :: Nil     // some syntax is meant to appeal to Haskell fans

def squared(x: Int) = x * x     // when required, types are capitalized and after colons

xs.map(squared)

xs.map(x => x + 1)              // short lambda syntax

xs.map(_ * 2)                   // even shorter for special cases (one argument, no type)

xs map {x => x + 1}             // dots and parentheses aren't always needed (again, like Haskel)

xs: List[Int] = List(1, 2, 3, 4, 5)
ys: List[Int] = List(1, 2, 3)
defined function squared
res47_3: List[Int] = List(1, 4, 9, 16, 25)
res47_4: List[Int] = List(2, 3, 4, 5, 6)
res47_5: List[Int] = List(2, 4, 6, 8, 10)
res47_6: List[Int] = List(2, 3, 4, 5, 6)

## Getting some data

These are the same CMS public data events as last time, now viewed as Scala objects.

(Also note that we're installing software and loading it interactively; one of Java's features is zero-hassle installation.)

In [49]:
import $ivy.`org.diana-hep:histogrammar_2.11:1.0.4`

import $ivy.$                                      

In [50]:
import org.dianahep.histogrammar.tutorial.cmsdata
val events = cmsdata.EventIterator().take(1000).toList

import org.dianahep.histogrammar.tutorial.cmsdata

events: List[cmsdata.Event] = List(
  Event(
    List(),
    List(
      Muon(
        4.8594961166381845,
        -30.2398738861084,
        137.7764892578125,
        141.13978576660156,
        -1,
        0.0
      )
...

Not coincidentally, the Scala functionals have mostly the same names as mine.

In [51]:
// sequential calculation
events.flatMap(_.muons).map(_.pt).take(10)

res50: List[Double] = List(
  30.62784150336687,
  31.641719444653546,
  25.997956733809474,
  40.514054143148,
  36.94010454322916,
  24.424106610105476,
  39.758086508574195,
  36.73538886104245,
  69.39617109916814,
  46.84325839229087
)

In [52]:
// parallel calculation (just add ".par" to use all your cores)
events.par.flatMap(_.muons).map(_.pt).take(10)

res51: collection.parallel.immutable.ParSeq[Double] = ParVector(30.62784150336687, 31.641719444653546, 25.997956733809474, 40.514054143148, 36.94010454322916, 24.424106610105476, 39.758086508574195, 36.73538886104245, 69.39617109916814, 46.84325839229087)

## Pattern matching

The most idiomatic Scala code compares values against expected patterns, like regular expressions, but for objects.

In [53]:
events.drop(30).take(20).map(_.muons) foreach {
    case List() =>
        println("no muons")
    
    case List(cmsdata.Muon(px, py, pz, energy, q, iso)) =>
        println("one %smuon E=%g".format(if (q < 0) "anti" else "", energy))
    
    case List(one, two) =>
        println("two muons, mass=%g".format((one + two).mass))
    
    case muons =>
        println("yikes! %d muons!".format(muons.size))
}

no muons
one muon E=30.0587
no muons
no muons
no muons
no muons
no muons
two muons, mass=88.9468
one antimuon E=157.522
no muons
one antimuon E=34.7609
no muons
no muons
one antimuon E=41.8070
one antimuon E=100.073
one muon E=161.699
no muons
no muons
no muons
no muons


## Immutable data

Remember `x = x + 1`? This mathematically false assertion (unless `x` is infinite) is tolerated in programming because we've gotten used to the idea that values change.

But if we really want the computer to be a mathematical scratchpad, values should be immutable (unchangeable once created).

This has some far-reaching consequences.

   * A programming _statement_ can do nothing other than cause change. If all data are immutable, there's no reason to have statements anymore (`for` loops become useless, only combinators like `map` are able to do anything).

   * Vastly simplifies _distributed data._ Whenever two computers see the same value, they are always looking at unliked copies of it: they can't change it without introducing inconsistencies. (And chasing inconsistencies in a distributed system is madness incarnate!)

   * Opens the door to _structural sharing,_ which improves performance over copying (next slide).

## Structural sharing

<table>
<tr style="background-color: white;"><td><span style="font-family: Lato, sans-serif; font-size: 35.84px">When <i>all</i> values are immutable, we can dramatically reduce the memory required for tree-like data structures by refusing to copy the ones that don't change in a transformation.</span></td><td style="width: 600px;"><img src="http://2.bp.blogspot.com/_r-NJO1NMiu4/TRA69XdCU8I/AAAAAAAAAnM/Re0VElAeLc4/s1600/ds_2_new.gif" style="margin-left: auto; margin-right: auto; width: 100%"></td></tr>
</table>

Totally immutable with structural sharing is a different sweet spot for performance than traditional transform-in-place, but it's often safer and easier to parallelize.

In [54]:
var identifier = 'A'

def message(id: Char) =
    if (id.toByte > 'G'.toByte)
        "    <-- new node"
    else
        ""

object TreeList {
    def apply[T](values: T*): TreeList[T] = {
        val (value, children) = values.toList match {
            case Nil => throw new Exception("cannot be empty")
            case one :: Nil => (one, List())
            case first :: rest =>
                val (left, right) = rest.splitAt(rest.size / 2)
                (first, List(left, right).flatMap({
                    case Nil => List()
                    case x => List(TreeList(x: _*))
                }))
        }

        new TreeList(value, children)
    }
}
class TreeList[T](val value: T, val children: List[TreeList[T]]) {
    val id = identifier
    identifier = (identifier.toByte + 1).toChar

    def toString(indent: String): String = {
        val prefix = "\n%s%s: value %s%s".format(indent, id, value, message(id))
        val subtrees = children.map(_.toString(indent + "    "))
        (prefix :: subtrees).mkString
    }
    override def toString() = toString("")

    def size: Int = 1 + children.map(_.size).sum
    
    def toList: List[T] = value +: children.flatMap(_.toList)
    
    def get(index: Int): T = index match {
        case 0 => value
        case i if i - 1 < children.head.size => children.head.get(i - 1)
        case i => children.last.get(i - 1 - children.head.size)
    }
    
    def inserted(index: Int, newval: T): TreeList[T] = index match {
        case 0 =>
            new TreeList(value, List(new TreeList(newval, children)))
        case i if i - 1 < children.head.size =>
            new TreeList(value, children.head.inserted(i - 1, newval) :: children.tail)
        case i =>
            new TreeList(value, List(children.head, children.last.inserted(i - 1 - children.head.size, newval)))
    }
}

identifier: Char = 'A'
defined function message
defined object TreeList
defined class TreeList

In [55]:
identifier = 'A'
val xs = TreeList(0, 1, 2, 3, 4, 5, 6)
xs.toList

xs: TreeList[Int] = 
G: value 0
    C: value 1
        A: value 2
        B: value 3
    F: value 4
        D: value 5
        E: value 6
res54_2: List[Int] = List(0, 1, 2, 3, 4, 5, 6)

In [56]:
val ys = xs.inserted(5, 999)
ys.toList

ys: TreeList[Int] = 
K: value 0    <-- new node
    C: value 1
        A: value 2
        B: value 3
    J: value 4    <-- new node
        I: value 5    <-- new node
            H: value 999    <-- new node
        E: value 6
res55_1: List[Int] = List(0, 1, 2, 3, 4, 5, 999, 6)

## The end!

Thanks for listening!

(I can't believe we'd have any more time than this, but if we do, I'll answer questions or try ad-libbing some examples or challenges.)